In [1]:
"""安装paddlex"""
! pip install paddlex==2.0.0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 952kB 7.1MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 45.2MB/s eta 0:00:01
     |████████████████████████████████| 1.5MB 24.7MB/s eta 0:00:01
     |████████████████████████████████| 296kB 45.7MB/s eta 0:00:01
     |████████████████████████████████| 153kB 470kB/s eta 0:00:01
     |████████████████████████████████| 6.8MB 4.7MB/s eta 0:00:01
     |████████████████████████████████| 112kB 9.1MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 51kB 221kB/s eta 0:00:011
     |████████████████████████████████| 286kB 6.9MB/s eta 0:00:01
     |████████████████████████████████| 102kB 17.5MB/s ta 0:00:01
     |████████████████████████████████| 102kB 7.0MB/s ta 0:00:011
  Created wheel for pycocotools: filename=pycocotools-2.0.4-cp37-cp37m-linux_x86_64.

In [1]:
"""导入包"""
import paddlex as pdx
import random
import numpy as np
import paddle

from paddlex import transforms as T

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/distributed/parallel.py:136: UserWarning: Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything.
  "Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything."


[01-20 19:58:44 MainThread @utils.py:79] WRN paddlepaddle version: 2.2.1. The dynamic graph version of PARL is under development, not fully tested and supported


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pyarrow/pandas_compat.py:1027: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  'floating': np.float,
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of

In [2]:
"""定义数据处理方式"""
train_transforms = T.Compose([
    T.Resize(target_size=1024),
    T.MixupImage(alpha=1.5, beta=1.5, mixup_epoch=-20),
    T.RandomDistort(brightness_range=0.5, brightness_prob=0.5, contrast_range=0.5, contrast_prob=0.5, saturation_range=0.5, saturation_prob=0.5, hue_range=18, hue_prob=0.5, random_apply=True, count=4, shuffle_channel=False),
    T.RandomBlur(prob=0.5),
    T.Normalize(
        mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

eval_transforms = T.Compose([
    T.Resize(target_size=1024),
    T.Normalize(
        mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

In [3]:
"""直接加载eval_dataset，后面预测就不用传入transfroms了"""
train_dataset = pdx.datasets.SegDataset(
    data_dir='',
    file_list='train_path/all_data.txt',
    label_list='train_path/labels.txt',
    transforms=train_transforms,
    shuffle=True)

eval_dataset = pdx.datasets.SegDataset(
    data_dir='',
    file_list='train_path/val.txt',
    label_list='train_path/labels.txt',
    transforms=eval_transforms)

2022-01-20 19:58:59 [INFO]	1913 samples in file train_path/all_data.txt
2022-01-20 19:58:59 [INFO]	50 samples in file train_path/val.txt


In [ ]:
"""训练模型并保存，可视化训练过程"""
# 使用随机增强不要固定种子
# random.seed(2048)
# np.random.seed(2048)
# paddle.seed(2048)

num_classes = len(train_dataset.labels)
model = pdx.seg.DeepLabV3P(num_classes=num_classes, use_mixed_loss=True)
model.train(
    num_epochs=50,
    train_dataset=train_dataset,
    train_batch_size=4,
    eval_dataset=eval_dataset,
    learning_rate=0.1,
    lr_decay_power=0.8,
    save_interval_epochs=1,
    save_dir='model/deeplab_augument_alldata3',
    use_vdl=True)

W0120 10:01:51.502168   504 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0120 10:01:51.507787   504 device_context.cc:465] device: 0, cuDNN Version: 7.6.


2022-01-20 10:01:54 [INFO]	Downloading model.pdparams from https://bj.bcebos.com/paddleseg/dygraph/cityscapes/deeplabv3p_resnet50_os8_cityscapes_1024x512_80k/model.pdparams


100%|██████████| 157928/157928 [00:03<00:00, 44124.91KB/s]


2022-01-20 10:01:58 [INFO]	Loading pretrained model from model/deeplab_augument_alldata2/pretrain/model.pdparams
2022-01-20 10:01:59 [WARNING]	[SKIP] Shape of pretrained params head.decoder.conv.weight doesn't match.(Pretrained: [19, 256, 1, 1], Actual: [3, 256, 1, 1])
2022-01-20 10:01:59 [WARNING]	[SKIP] Shape of pretrained params head.decoder.conv.bias doesn't match.(Pretrained: [19], Actual: [3])
2022-01-20 10:01:59 [INFO]	There are 358/360 variables loaded into DeepLabV3P.
2022-01-20 10:02:03 [INFO]	[TRAIN] Epoch=1/50, Step=2/478, loss=0.425905, lr=0.099997, time_each_step=1.94s, eta=12:54:17
2022-01-20 10:02:06 [INFO]	[TRAIN] Epoch=1/50, Step=4/478, loss=0.496119, lr=0.099990, time_each_step=1.26s, eta=8:20:18
2022-01-20 10:02:08 [INFO]	[TRAIN] Epoch=1/50, Step=6/478, loss=0.562343, lr=0.099983, time_each_step=1.26s, eta=8:20:38
2022-01-20 10:02:11 [INFO]	[TRAIN] Epoch=1/50, Step=8/478, loss=0.230614, lr=0.099977, time_each_step=1.26s, eta=8:20:52
2022-01-20 10:02:13 [INFO]	[TRAIN

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:253: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.int32, but right dtype is paddle.bool, the right dtype will convert to paddle.int32
  format(lhs_dtype, rhs_dtype, lhs_dtype))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:253: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.int64, but right dtype is paddle.bool, the right dtype will convert to paddle.int64
  format(lhs_dtype, rhs_dtype, lhs_dtype))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/dygraph/math_op_patch.py:253: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.int64, but right dtype is paddle.int32, the right dtype will convert to paddle.int64
  format(lhs_dtype, rhs_dtype, lhs_dtype))


2022-01-20 10:12:20 [INFO]	[EVAL] Finished, Epoch=1, miou=0.790595, category_iou=[0.96693116 0.6539901  0.7508646 ], oacc=0.967731, category_acc=[0.9819752  0.85272247 0.8535073 ], kappa=0.840675, category_F1-score=[0.98318761 0.79080294 0.85770729] .
2022-01-20 10:12:20 [INFO]	Model saved in model/deeplab_augument_alldata2/best_model.
2022-01-20 10:12:20 [INFO]	Current evaluated best model on eval_dataset is epoch_1, miou=0.790595293045044
2022-01-20 10:12:21 [INFO]	Model saved in model/deeplab_augument_alldata2/epoch_1.
2022-01-20 10:12:24 [INFO]	[TRAIN] Epoch=2/50, Step=2/478, loss=0.119015, lr=0.098393, time_each_step=1.8s, eta=11:49:10
2022-01-20 10:12:27 [INFO]	[TRAIN] Epoch=2/50, Step=4/478, loss=0.086192, lr=0.098387, time_each_step=1.29s, eta=8:27:20
2022-01-20 10:12:29 [INFO]	[TRAIN] Epoch=2/50, Step=6/478, loss=0.089094, lr=0.098380, time_each_step=1.29s, eta=8:27:9
2022-01-20 10:12:32 [INFO]	[TRAIN] Epoch=2/50, Step=8/478, loss=0.079278, lr=0.098373, time_each_step=1.28s, e